In [21]:
import os
import cv2
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.models import vgg16
import torch
import tqdm
import torch.nn as nn

In [15]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [41]:
# check device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [42]:

classes = os.listdir('Data/genres_original')
classes.sort()
class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
idx_to_class = {i: cls_name for i, cls_name in enumerate(classes)}

images = []
labels = []
for i, cls_name in enumerate(classes):
    path = os.path.join('Data/images_original', cls_name)
    for img_file in os.listdir(path):
        img_path = os.path.join(path, img_file)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = transform(img)
        images.append(img)
        labels.append(i)
images = torch.stack(images).to(device)
labels = torch.tensor(labels).to(device)
print(images.shape, labels.shape)

torch.Size([999, 3, 224, 224]) torch.Size([999])


In [43]:
from sklearn.model_selection import train_test_split

train_images, test_images, train_labels, test_labels = train_test_split(images, labels, shuffle=True, test_size=0.2, random_state=42)
print(train_images.shape, test_images.shape, train_labels.shape, test_labels.shape)

torch.Size([799, 3, 224, 224]) torch.Size([200, 3, 224, 224]) torch.Size([799]) torch.Size([200])


In [44]:

class MusicDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

In [47]:
dataset_train = MusicDataset(train_images, train_labels)
DataLoader_train = DataLoader(dataset_train, batch_size=50, shuffle=True)

In [48]:
import torch.nn as nn

class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x

In [49]:
model = vgg16(pretrained=True)

for param in model.parameters():
    param.requires_grad = False


model.classifier = nn.Sequential(
    nn.Linear(25088, 2048),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(2048, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 10),
    nn.LogSoftmax(dim=1)
)
model = model.to(device)
model

/Users/liutianhao/.pyenv/versions/3.8.18/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/liutianhao/.pyenv/versions/3.8.18/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [50]:

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

# 训练模型 
num_epochs = 30
patience = 5  # Number of epochs to wait for improvement before stopping
best_val_loss = float('inf')  # Initialize best validation loss
patience_counter = 0  # Counter to keep track of the number of epochs without improvement
log = {}


for epoch in range(num_epochs):
    model.train()
    run_loss = 0.0
    for images, labels in tqdm.tqdm(DataLoader_train):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        run_loss += loss.item()

    model.eval()
    with torch.no_grad():
        test_outputs = model(test_images)
        test_loss = criterion(test_outputs, test_labels)
        pred = torch.argmax(test_outputs, dim=1)
        acc = (pred == test_labels).sum().item() / len(test_labels)

        # save the loss and accuracy
        log[epoch] = {'loss_train': loss.item(), 'loss_val': test_loss.item(), 'accuracy_val': acc}
        print(f'Epoch {epoch}, loss_train: {loss.item() / len(DataLoader_train)}, loss_val: {test_loss.item()}, accuracy_val: {acc}')

    if test_loss < best_val_loss:
        best_val_loss = test_loss
        patience_counter = 0
        torch.save(model.state_dict(), 'find-tuned-vgg16.pth')
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f'Early stopping at epoch {epoch}')
            break

100%|██████████| 16/16 [01:13<00:00,  4.60s/it]


Epoch 0, loss_train: 0.11010483652353287, loss_val: 1.7232975959777832, accuracy_val: 0.315


100%|██████████| 16/16 [01:20<00:00,  5.02s/it]


Epoch 1, loss_train: 0.10858792811632156, loss_val: 1.3139467239379883, accuracy_val: 0.54


100%|██████████| 16/16 [01:09<00:00,  4.32s/it]


Epoch 2, loss_train: 0.05599985271692276, loss_val: 1.2362163066864014, accuracy_val: 0.525


 56%|█████▋    | 9/16 [00:44<00:35,  5.01s/it]